# Data Cleaning


1. Drop unused info from Inspection Data
2. Merge with yelp data
3. Merge all Yelp/Google review data together and remove irrelavent words
4. Transform last_inspection_date to days_since_last
5. Output to 01_train.csv

In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model
from sklearn import metrics
import seaborn as sns 
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [48]:
yelp = pd.read_csv('matched_yelp_data.csv')
yelp = yelp[['CAMIS','ADDRESS','ZIPCODE','yelp_price','yelp_rating','yelp_review_count', 'yelp_categories_a', 'yelp_categories_t','yelp_latitude', 'yelp_longitude']]
stop_words = set(stopwords.words('english')) 
stop_words.update(['rating','text','time','n','created','CUISINE','DESCRIPTION','yelp','categories','a','Name', 'dtype', 'object'])
def getUnique(x,y,z):
    res = set()
    res.update(concatStrings(x).split())
    res.update(concatStrings(y).split())
    res.update(concatStrings(z).split())
    return ' '.join(res)

def concatStrings(s):
    tokens = word_tokenize(''.join(re.sub(r"[^a-zA-Z]+", ' ', str(s).lower())))
    filtered_sentence = [w for w in tokens if not w in stop_words]          
    return ' '.join(filtered_sentence)
reviews = pd.read_csv('dataset_v7.csv')
reviews = reviews[['CAMIS','Signs of Rodents','yelp_reviews','gmap_reviews']]
reviews['yelp_reviews_concat'] = reviews['yelp_reviews'].apply(concatStrings)
reviews['gmap_reviews_concat'] = reviews['gmap_reviews'].apply(concatStrings)
reviews['combined_reviews'] = reviews[['yelp_reviews_concat', 'gmap_reviews_concat']].apply(lambda x: ''.join(x), axis=1)
reviews = reviews[['CAMIS','Signs of Rodents','combined_reviews']]


def data_cleaning(df):
    df = pd.merge(left=df, right=yelp,how='left',left_on='CAMIS', right_on='CAMIS')
    df = pd.merge(left=df, right=reviews,how='left',left_on='CAMIS', right_on='CAMIS')

    df['categories'] = df.apply(lambda x: getUnique(x['CUISINE DESCRIPTION'],x['yelp_categories_a'],x['yelp_categories_t']),axis=1)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['last_inspection_date'] = pd.to_datetime(df['last_inspection_date'])
    df['Days_since_last'] = df.apply(lambda x: (x['DATE']-x['last_inspection_date']).days, axis=1)
    df[df['Days_since_last'].isnull()]['Days_since_last'] = 0
    df.drop(['CUISINE DESCRIPTION', 'yelp_categories_a','yelp_categories_t','last_inspection_date'],axis=1,inplace=True)
    df.sort_values(['CAMIS','DATE'],ascending = [1,0],inplace = True)
    df.reset_index(drop=True)
    return df

#Clean testset
test = pd.read_csv('01_CAMIS_TESTSET_WITH_PREV.csv')
df['CAMIS'] = df['CAMIS2']
df['DATE'] = df['DATE2']
df.drop(['Unnamed: 0','index','CAMIS2.1','CAMIS2','DATE2','CRITICAL FLAG','VIOLATION CODE','ACTION','VIOLATION DESCRIPTION','SCORE'],axis=1,inplace=True)


# Clean trainingset
training_base = pd.read_csv('01_CAMIS_TRAINING__WITH_PREV.csv')
tmp1 = training_base[training_base['TARGET'] > 0].sample(n=11186)
tmp2 = training_base[training_base['TARGET'] == 0]
training = pd.concat([tmp1,tmp2])
df.drop(['Unnamed: 0','index','CAMIS2','DATE2','CRITICAL FLAG','VIOLATION CODE','ACTION','VIOLATION DESCRIPTION','SCORE'],axis=1,inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
data_cleaning(test)to_csv('01_test.csv')
data_cleaning(training)to_csv('01_training.csv')

In [54]:
training = pd.read_csv('01_training.csv')
training

,Unnamed: 0,CAMIS,DATE,last_crit_flag,last_violation_code,last_score,TARGET,ADDRESS,ZIPCODE,yelp_price,yelp_rating,yelp_review_count,yelp_latitude,yelp_longitude,Signs of Rodents,combined_reviews,categories,Days_since_last
0,11186,30075445.0,2018-05-11,1.0,06D10F,7.0,0,"1007,MORRIS PARK AVE",10462.0,2.0,4.0,37.0,40.848390,-73.856010,NaN,NaN,desserts bakery bakeries,358.0
1,11187,30112340.0,2016-04-12,1.0,10B04A,12.0,0,"469,FLATBUSH AVENUE",11225.0,1.0,2.0,37.0,40.662952,-73.961753,NaN,NaN,food hamburgers hotdogs fast burgers,341.0
2,8350,40356731.0,2018-05-16,1.0,08A04L,10.0,1,"1839,NOSTRAND AVENUE",11226.0,1.0,4.5,30.0,40.640830,-73.948350,102.0,Despite close proximity place residence I many...,yogurt cream icecream gelato ices frozen ice,328.0
3,11188,40356731.0,2015-06-26,NaN,NaN,NaN,0,"1839,NOSTRAND AVENUE",11226.0,1.0,4.5,30.0,40.640830,-73.948350,102.0,Despite close proximity place residence I many...,yogurt cream icecream gelato ices frozen ice,NaN
4,6555,40359705.0,2016-07-08,3.0,06F02B06C,19.0,1,"1310,SURF AVENUE",11224.0,1.0,3.5,1104.0,40.575390,-73.981510,22.0,I mean NATHANS FAMOUS go wrong nCame around co...,dogs food hotdog hot hotdogs fast,37.0
5,9271,40359705.0,2015-04-14,2.0,10A02G02B,18.0,1,"1310,SURF AVENUE",11224.0,1.0,3.5,1104.0,40.575390,-73.981510,22.0,I mean NATHANS FAMOUS go wrong nCame around co...,dogs food hotdog hot hotdogs fast,13.0
6,11189,40360045.0,2018-11-27,1.0,04N10F08A09C,13.0,0,"705,KINGS HIGHWAY",11223.0,NaN,4.5,4.0,40.606472,-73.965415,NaN,NaN,caterers catering kosher jewish,365.0
7,11190,40360076.0,2017-03-16,1.0,06C,5.0,0,"203,CHURCH AVENUE",11218.0,1.0,3.5,25.0,40.643479,-73.978185,NaN,NaN,yogurt cream icecream gelato ices frozen ice,401.0
8,5752,40361322.0,2018-03-28,2.0,06D06B,11.0,1,"26515,HILLSIDE AVENUE",11004.0,2.0,3.0,29.0,40.738680,-73.703230,NaN,NaN,yogurt cream icecream gelato ices frozen ice,447.0
9,9012,40361618.0,2016-02-11,3.0,02B06D06C,18.0,1,"12908,20 AVENUE",11356.0,1.0,5.0,9.0,40.781479,-73.839311,NaN,NaN,sandwiches delis delicatessen,14.0


In [55]:
test = pd.read_csv('01_test.csv')
test

,Unnamed: 0,last_crit_flag,last_violation_code,last_score,TARGET,CAMIS,DATE,ADDRESS,ZIPCODE,yelp_price,yelp_rating,yelp_review_count,yelp_latitude,yelp_longitude,Signs of Rodents,combined_reviews,categories,Days_since_last
0,694,2.0,06D10B06C,12.0,1,40364576.0,2018-11-28,"311,WEST 51 STREET",10019.0,2.0,3.5,299.0,40.763405,-73.986214,22.0,french food pretentious overpriced case tout v...,french,141.0
1,175,1.0,10F06D,10.0,1,40365361.0,2018-03-27,"15,WEST 43 STREET",10036.0,NaN,3.5,49.0,40.754660,-73.981320,17.0,spending pcny events really appreciated staff ...,clubs american social,404.0
2,815,6.0,06C02H02B08A04M02I08C04L,40.0,1,40366586.0,2018-03-20,"7124,MAIN STREET",11367.0,1.0,3.0,20.0,40.727249,-73.822397,NaN,NaN,pizza kosher jewish,132.0
3,208,2.0,06D10F06F,13.0,1,40368338.0,2018-05-17,"1583,2 AVENUE",10028.0,1.0,3.5,58.0,40.775480,-73.953800,NaN,NaN,american bars,344.0
4,369,1.0,10F06C10E,13.0,1,40368526.0,2018-10-25,"193,FROST STREET",11211.0,2.0,4.0,137.0,40.718387,-73.943491,57.0,place blocks house never tried usual pasta spo...,italian wine seafood bars,168.0
5,71,2.0,08A02G04L10B,23.0,1,40369775.0,2018-10-02,"359,WEST BROADWAY",10013.0,2.0,3.5,565.0,40.722961,-74.003205,15.0,hotel concierge recommended cupping room reser...,cocktailbars brunch traditional bars breakfast...,11.0
6,770,1.0,02B10F,10.0,1,40369849.0,2018-04-10,"6132,SPRINGFIELD BOULEVARD",11364.0,2.0,3.5,178.0,40.750813,-73.755664,3.0,went back favorite pizza place gino today hung...,pizza italian,210.0
7,725,2.0,06C06D10B,12.0,1,40369958.0,2018-08-06,"210,EAST 46 STREET",10017.0,4.0,3.5,842.0,40.753030,-73.972329,4.0,joey bagels go spot loved chateaubriand fn guy...,steak steakhouses,385.0
8,895,1.0,06D10F,9.0,0,40371718.0,2018-04-30,"500,GRAND STREET",10002.0,1.0,3.0,56.0,40.714963,-73.982653,45.0,love local neighborhood gem going since kid st...,american diners breakfast brunch,350.0
9,225,1.0,08A04L,9.0,1,40372112.0,2018-07-18,"7717,THIRD AVENUE",11209.0,2.0,3.5,110.0,40.629597,-74.028422,49.0,breakfast lunch brunch food delicious choice u...,italian american new newamerican,146.0


In [59]:
len(training.columns.tolist())

18

In [60]:
len(test.columns.tolist())

18